# Indexing unknown secondary phases

This notebook gives a demo for a couple of methods for selecting a grain and indexing it.

1: select a peak, find the Friedel pair, use the pair to locate a position

2: just select a position in space.

Then filter spots by position, and/or, by using a selected peak to generate lattice translations and checking for translational symmetry.

Subsets of selected peaks are then indexing using index_unknown.py from ImageD11

Last modified 15/11/2024

In [ ]:
import os, sys
# USER: You can change this location if you want
exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())
PYTHONPATH = setup_ImageD11_from_git()

In [ ]:
%matplotlib ipympl
import numpy as np
import matplotlib.pyplot as plt
import ImageD11.sinograms.dataset
from ImageD11.nbGui.interactive_peakselect import FriedelPairSelector
import ImageD11.nbGui.nb_utils as utils
import ImageD11.peakselect
import scipy.spatial
import ImageD11.transformer
import ImageD11.indexing
import ImageD11.sinograms.geometry

In [ ]:
def find_overlaps_3d( cf, pkids, gtol = 5e-3 ):
    """
    cf = columnfile with g-vectors
    pkids = peaks to use for offsetting your data and overlapping
    
    This shifts the g-vectors by the gvector of pkid and checks for overlaps
    
    returns (peaks_having_overlaps, distances used)
    """
    g3 = np.transpose( (cf.gx, cf.gy, cf.gz) )
    kd0 = scipy.spatial.cKDTree( g3 )
    pks = []
    distances = []
    for pkid in pkids:
        pk1 = np.argmin( abs( cf.spot3d_id - pkid ) )
        for o in (-1, 1):
            kd1 = scipy.spatial.cKDTree( g3 + o * g3[pk1] ) # plus or minus
            sd1 = kd0.sparse_distance_matrix( kd1, max_distance=gtol ).tocoo()
            #print(pkid, pk1, o, g3.shape, sd1.nnz)
            pks.append( sd1.row )
            pks.append( sd1.col )
            distances.append( sd1.data )
    return np.unique( np.concatenate( pks ) ), np.concatenate( distances )



def run_index_unknown(gid, cf, frac=0.2, tol=0.05):
    """
    gid = string to name files
    cf = colfile to index
    frac = fraction of peaks you want to index
    tol = hkl tolerance
    """
    tr = ImageD11.transformer.transformer()
    tr.colfile = cf
    tr.parameterobj = cf.parameters
    # need to have cell params to save gves
    tr.parameterobj.set('cell_lattice_[P,A,B,C,I,F,R]','P')# integer not backwards compatible
    tr.savegv( f'gr{gid}.gve' )
    !index_unknown.py -g gr{gid}.gve -m 40 --fft -t {tol} -f {frac} -o {gid}.ubi -k 1
    if os.path.exists(f'{gid}.ubi'):
        fixhandedness( f'{gid}.ubi' ) # the script on path might not be the one in git
    
def fixhandedness( ubifile ):
    ubis = ImageD11.indexing.readubis( ubifile )
    for i in range(len(ubis)):
        if np.linalg.det( ubis[i] ) < 0:
            ubis[i][-1] = -ubis[i][-1]
        assert np.linalg.det( ubis[i] ) > 0
    ImageD11.indexing.write_ubi_file(  ubifile, ubis )
    
def choose_peaks(cf, g, tol=1e-4):
    """
    get the peaks a grain indexes using g error tol
    Belongs somewhere else (grain method?)
    """
    gve = (cf.gx, cf.gy, cf.gz)
    hkl = g.ubi.dot( gve )
    gcalc = g.ub.dot( np.round( hkl ) )
    gerr = ((gcalc - gve )**2).sum(axis=0)
    return gerr < tol

def plot_indexing( cf, ubifiles ):
    gl = []
    for f in ubifiles:
        gl += ImageD11.grain.read_grain_file( f ) 
    fig = plt.figure()
    ax = fig.add_subplot( 1, 2, 1, projection='3d', proj_type='ortho')
    ax2 = fig.add_subplot(1, 2, 2)
    ax.plot( cf.gx, cf.gy, cf.gz, '.', ms = 1)
    ax2.plot( cf.omega, cf.dty, ".", ms=1 )
    for i,g in enumerate(gl):
        indexed = choose_peaks( cf, g )
        ax.scatter( cf.gx[indexed], cf.gy[indexed], cf.gz[indexed], s=3, c='rgbmyk'[i] )
        for j in range(3):
            ax.plot( [0,g.ub[0,j]],
                     [0,g.ub[1,j]],
                     [0,g.ub[2,j]],
                      '-'+'rgb'[j] )
        ax2.plot( cf.omega[ indexed ], cf.dty[indexed], "+"+'rgbmyk'[i] )    

In [ ]:
dset_file = 'path/to/dataset.h5'
ds = ImageD11.sinograms.dataset.load(dset_file)
print(ds)

In [ ]:
ds.phases = ds.get_phases_from_disk()
ds.phases.unitcells

In [ ]:
major_phase_strs = ['Fe', 'Au']
major_phase_unitcells = [ds.phases.unitcells[mps] for mps in major_phase_strs]
print(major_phase_unitcells)

In [ ]:
cf_4d = ds.get_cf_4d()
ds.update_colfile_pars(cf_4d, major_phase_strs[0] )

In [ ]:
# remove the peaks that are not found on more than one frame
cf_4d.filter( cf_4d.npk2d > 2 )

In [ ]:
# powder patterns
pbins = 5000
hsI, dsbinedges = np.histogram( cf_4d.ds, weights=cf_4d.sum_intensity, bins=np.linspace(0, cf_4d.ds.max(), pbins))
hs1, dsbinedges = np.histogram( cf_4d.ds, bins=np.linspace(0, cf_4d.ds.max(), pbins))
dsbincens = 0.5*(dsbinedges[1:]+dsbinedges[:-1])

In [ ]:
# filter out peaks from major phases
major_phase_dstol = 0.004
cf_4d = ImageD11.peakselect.remove_peaks_from_phases(cf_4d, major_phase_dstol, major_phase_unitcells)

In [ ]:
# cring = columnfile of things not indexed by the main rings
dsmax = 1.0
cring = cf_4d.copyrows(cf_4d.ds < dsmax)

In [ ]:
f, ax = plt.subplots(constrained_layout=True, figsize=(10,7))
ax.plot( dsbincens, hsI/hsI.max() , '-', label='Pre-filtration histogram')
for inc, major_phase_unitcell in enumerate(major_phase_unitcells):
    ax.vlines(major_phase_unitcell.ringds, 1e-3*(2**inc), 1e-3*(2**inc)*2, color=plt.color_sequences['tab20'][inc+2], label=major_phase_unitcell.name)
ax.plot( cring.ds, cring.sum_intensity/cring.sum_intensity.max(), ".", ms=1, label='Surviving peaks' )
ax.set(xlabel=r'$d^{*}~(\AA^{-1})$', ylabel='Fractional intensity', yscale='log', xlim=(0.1,1), ylim=(1e-5,1))
ax.legend()
plt.show()

## impurity phase method 1 : pick a peak

Select a not-indexed ring in two theta that appears on more than one frame (npk2d > 1)

This is plotted on the left as a sinogram.

On the right you have all the non-indexed peaks.

When the peak is selected, the code looks for the Friedel pair and tries to fit a grain position.

We assume the y0 is 0 (otherwise you need to input this).

In [ ]:
# from the plot above, choose a ring_dstar that represents a ring of an unknown phase

ring_dstar = 0.32028
ring_dstol = 0.003

m = abs(cring.ds - ring_dstar) < ring_dstol

cpk = cring.copyrows( m )
cpk.sortby( 'sum_intensity' )
cring.sortby( 'sum_intensity' )

In [ ]:
# Position where the beam goes through the rotation axis
y0 = 0.0

In [ ]:
selector_gui = FriedelPairSelector(cring, cpk, y0)

In [ ]:
p1, p2, xy, peak_ycalc = selector_gui.get_selection()

In [ ]:
dtyrange = ds.ystep * 5
f,ax = plt.subplots(constrained_layout=True, figsize=(7,4))
ax.hist(cring.dty - peak_ycalc, bins = np.arange(-dtyrange,dtyrange,ds.ystep))
ax.set(title='dty error histogram', xlabel='dty', ylabel='frequency')
plt.show()

In [ ]:
# now we select the peaks from the sinogram based on that position in space
dty_cut = 2.5 # from the last plot

ymask = abs(peak_ycalc - cring.dty)<dty_cut
cgrain = cring.copyrows( ymask )
fig = plt.figure(constrained_layout=True)
ax = fig.add_subplot( projection='3d', proj_type='ortho')
ax.scatter( cgrain.gx, cgrain.gy, cgrain.gz, s=1 )
ax.set(title='Scattering vectors, can you see the lattice yet?')
plt.show()

In [ ]:
pks, dists = find_overlaps_3d( cring, (cpk.spot3d_id[p1], cpk.spot3d_id[p2] ), gtol=0.01 )
fig, ax = plt.subplots(constrained_layout=True)
ax.hist(dists, bins=20)
ax.set(xlabel='G-vector distances', ylabel='Counts')
plt.show()

In [ ]:
gtol = 0.002 # from the last plot
pks, dists = find_overlaps_3d( cgrain, (cpk.spot3d_id[p1], cpk.spot3d_id[p2] ), gtol=gtol )

fig = plt.figure(constrained_layout=True)
a = fig.add_subplot(1,1,1, projection='3d', proj_type='ortho')
a.scatter(cgrain.gx,cgrain.gy,cgrain.gz,s=1,alpha=0.4)
a.scatter(cgrain.gx[pks],cgrain.gy[pks],cgrain.gz[pks],s=10)
a.set(title='Lattice overlap detection')
plt.show()

In [ ]:
# indexing using the lattice we found
run_index_unknown(0, cgrain.copyrows(pks))

In [ ]:
plot_indexing( cgrain, ("0.ubi",))

## Position selection on the sinogram

In [ ]:
# make a sinogram and recon
nbins_angle = min( len( ds.obincens), int(len(ds.ybincens)*1.5) )
sino, obinedge, ybinedge = np.histogram2d( cring.omega, cring.dty, weights = np.log(cring.sum_intensity),
                      bins = (np.linspace(ds.obinedges.min(), ds.obinedges.max(), nbins_angle), ds.ybinedges) )
obincen = 0.5*(obinedge[:-1] + obinedge[1:])
recon = ImageD11.sinograms.roi_iradon.run_iradon(sino.T, obincen, filter_name='shepp-logan', 
                                             workers=ImageD11.cImageD11.cores_available(), shift=-y0/ds.ystep, pad=0)

In [ ]:
# plot the sino and recon
fig, ax = plt.subplots(1,2,figsize=(12,6))
ax[0].pcolormesh( obinedge,
                  ybinedge,
                  sino.T);
ax[1].pcolormesh( -ds.ybinedges,ds.ybinedges, recon, vmin=0, vmax = recon.max()/2 )
location, = ax[1].plot( [],[], 'ow', mfc='none', ms=20)
ax[1].set(xlim=(ds.ybinedges[-1],ds.ybinedges[0]),
          xlabel='laboratory Y', ylabel='laboratory X')
ax[0].set(title='sinogram')
ax[1].set(title='click on the place you want')

om = np.linspace( cring.omega.min(), cring.omega.max(), 90 )
fitline1, = ax[0].plot( om, np.zeros_like(om), 'w-')
fitline2, = ax[0].plot( om, np.zeros_like(om), 'w-')

ycalcall = None
pos = None
def onclick( evt ):
    if evt.inaxes == ax[1]:
        y = evt.xdata
        x = evt.ydata
        ycalc = ImageD11.sinograms.geometry.dtycalc( om, x, y, y0 )
        global ycalcall, pos
        pos = y,x
        ycalcall = ImageD11.sinograms.geometry.dtycalc( cring.omega, x, y, y0 )
        fitline1.set_ydata( ycalc + 1 )
        fitline2.set_ydata( ycalc - 1 )
        location.set_xdata( [y,] )
        location.set_ydata( [x,] )
        fig.canvas.draw_idle()
    
fig.canvas.mpl_connect( 'button_press_event', onclick );

In [ ]:
print("Your click was",pos)

In [ ]:
ytol = 0.5
ymask = abs(ycalcall - cring.dty)<(ytol*ds.ystep)
cgrain2 = cring.copyrows( ymask )
fig = plt.figure()
ax = fig.add_subplot( projection='3d', proj_type='ortho')
ax.scatter( cgrain2.gx, cgrain2.gy, cgrain2.gz, s=1 );

In [ ]:
# index using all peaks
run_index_unknown( 1, cgrain2 )
indexed = choose_peaks( cgrain2, ImageD11.grain.read_grain_file( '1.ubi')[0] )
# re-index using the peaks indexed
run_index_unknown( 2, cgrain2.copyrows( indexed ) )

In [ ]:
plot_indexing( cgrain2, ('2.ubi',))

In [ ]:
# A final powder pattern to compare to our grains
h, _ = np.histogram( cring.ds, weights=np.log(cring.sum_intensity), bins=dsbinedges )

In [ ]:
cells = [ ImageD11.unitcell.unitcell( ImageD11.grain.read_grain_file( '%d.ubi'%(i) )[0].unitcell, 'P' )
          for i in range(3) ]
for c in cells:
    c.makerings(1)
    print(c.lattice_parameters)

In [ ]:
fig, ax = plt.subplots(figsize=(12,5), constrained_layout=True)
ax.plot( cring.ds, cring.sum_intensity, 'b.', ms=1)
ax.plot( cf_4d.ds, cf_4d.sum_intensity, 'g,', alpha=0.5)
for inc, cell in enumerate(cells):
    ax.vlines(cell.ringds, 1e3*(2**inc), 1e3*(2**inc)*2, color=plt.color_sequences['tab20'][inc+5], label=("%.4f "*6)%tuple(cell.lattice_parameters) )
for inc, major_phase_unitcell in enumerate(major_phase_unitcells):
    ax.vlines(major_phase_unitcell.ringds, 1e3*(2**(len(cells)+inc)), 1e3*(2**(len(cells)+inc))*2, color=plt.color_sequences['tab20'][inc], label=major_phase_unitcell.name)
ax.set(xlim=(0.1, cring.ds.max()), xlabel=r'$d^{*}~(\AA^{-1})$', ylabel='Intensity', yscale='log')
ax.legend(loc='upper left')
plt.show()

You can check for higher symmetry at https://www.cryst.ehu.es/cryst/lattice.html